<a href="https://colab.research.google.com/github/ZodiaX1013/039_JaydeepGamit/blob/main/ML6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

import numpy as np

In [2]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert to float32.
x_train = np.array(x_train, np.float32)
x_test = np.array(x_test, np.float32)


# Flatten images to 1-D vector of 784 features (28*28).
num_features=784
x_train = x_train.reshape(-1,num_features)
x_test = x_test.reshape(-1,num_features)


# Normalize images value from [0, 255] to [0, 1].
x_train = x_train / 255
x_test = x_test /255


11501568/11490434 [==============================] - 0s 0us/step


In [3]:
print(f'training data {x_train.shape}')
print(f'test data {x_test.shape}')
print(f'training label {y_train.shape}')
print(f'testing label {y_train.shape}')

training data (60000, 784)
test data (10000, 784)
training label (60000,)
testing label (60000,)


In [4]:
# MNIST dataset parameters.

num_classes = 10 # 0 to 9 digits

num_features = 784 # 28*28

# Training parameters.

learning_rate = 0.01

training_steps = 1000

batch_size = 256

display_step = 50

In [5]:
# Use tf.data API to shuffle and batch data.
num_batches = int(x_train.shape[0]/batch_size)

train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))

train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [6]:
# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.

W = tf.Variable(np.random.randn(784,10).astype(np.float32))
# X = tf.placeholder(tf.float32,[None,784])
# Y = tf.placeholder(tf.float32, [None,10])

# Bias of shape [10], the total number of classes.
B = tf.Variable(np.random.randn(10).astype(np.float32))

In [7]:
# Logistic regression (Wx + b).

def logistic_regression(x):

    # Apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(tf.add(tf.matmul(x,W),B))
    

# Cross-Entropy loss function.

def cross_entropy(y_pred, y_true):

    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)
    

    # Clip prediction values to avoid log(0) error.

    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)    

    # Compute cross-entropy.
    ce_loss = tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))
    return ce_loss

In [8]:
# Accuracy metric.

def accuracy(y_pred, y_true):

  # Predicted class is the index of the highest score in prediction vector (i.e. argmax).

  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))

  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [9]:
# Optimization process. 

optimizer = tf.optimizers.SGD(learning_rate)

def run_optimization(x, y):

# Wrap computation inside a GradientTape for automatic differentiation.

    with tf.GradientTape() as g:

        pred = logistic_regression(x)

        loss = cross_entropy(pred, y)

    # Compute gradients.

    gradients = g.gradient(loss, [W, B])


    # Stochastic gradient descent optimizer.

    
    
    # Update W and b following gradients.

    optimizer.apply_gradients(zip(gradients, [W, B]))

In [10]:
# Run training for the given number of steps.

for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):

    # Run the optimization to update W and b values.

    run_optimization(batch_x, batch_y)

    

    if step % display_step == 0:

        #Obtain Predictions
        pred = logistic_regression(batch_x)
        #Ccompute loss
        loss = cross_entropy(pred, batch_y)
        #Compute Accuracy
        acc = accuracy(pred, batch_y)
        #print accuracy
        print(f"step: {step}, loss: {loss}, accuracy: {acc}")

step: 50, loss: 234.72024536132812, accuracy: 0.796875
step: 100, loss: 145.605224609375, accuracy: 0.859375
step: 150, loss: 115.4985580444336, accuracy: 0.89453125
step: 200, loss: 137.80514526367188, accuracy: 0.8671875
step: 250, loss: 121.01780700683594, accuracy: 0.890625
step: 300, loss: 132.30264282226562, accuracy: 0.87890625
step: 350, loss: 151.6096649169922, accuracy: 0.8671875
step: 400, loss: 102.3332748413086, accuracy: 0.8984375
step: 450, loss: 105.82766723632812, accuracy: 0.9140625
step: 500, loss: 60.79877471923828, accuracy: 0.9296875
step: 550, loss: 135.27911376953125, accuracy: 0.89453125
step: 600, loss: 82.4313735961914, accuracy: 0.90234375
step: 650, loss: 109.4801025390625, accuracy: 0.8828125
step: 700, loss: 70.33497619628906, accuracy: 0.91796875
step: 750, loss: 105.87158203125, accuracy: 0.8828125
step: 800, loss: 121.59407806396484, accuracy: 0.91796875
step: 850, loss: 61.45993423461914, accuracy: 0.921875
step: 900, loss: 100.4066162109375, accuracy

In [11]:
# Test model on validation set.
pred = logistic_regression(x_test)
accu = accuracy(pred, y_test)
print(f"Test Accuracy: {accu}")

Test Accuracy: 0.9035000205039978
